# model.ipynb
This notebook:
- Creates a small sample dataset
- Trains a RandomForest model
- Generates predictions.json for dashboard

In [1]:
# Install packages (optional)
# !pip install pandas numpy scikit-learn

In [3]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

os.makedirs("data", exist_ok=True)

csv_path = "data/locations.csv"

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    rng = np.random.RandomState(42)
    n = 20
    df = pd.DataFrame({
        'id': np.arange(n),
        'latitude': 15 + rng.randn(n) * 2,
        'longitude': 78 + rng.randn(n) * 2,
        'rainfall_mm': np.abs(rng.randn(n) * 100),
        'soil_moisture': np.clip(rng.rand(n), 0, 1),
        'elevation': np.abs(rng.randn(n) * 50 + 200)
    })
    df['label'] = ((df['rainfall_mm'] > 70) & (df['elevation'] < 220)).astype(int)
    df.to_csv(csv_path, index=False)

df.head()

,location,latitude,longitude,rainfall,river_level,humidity
0,Hyderabad,17.3850,78.4867,220,4.2,82
1,Vijayawada,16.5062,80.6480,180,3.8,75
2,Chennai,13.0827,80.2707,300,5.5,90
3,Mumbai,19.0760,72.8777,350,6.1,88
4,Bangalore,12.9716,77.5946,120,2.5,65


In [6]:
import pandas as pd
import numpy as np
import json
import os

df = pd.read_csv("data/locations.csv")

# Use real columns from your CSV
features = ['rainfall', 'river_level', 'humidity']

df = df.dropna(subset=features)

# Simple rule-based risk score
risk_score = (
    (df['rainfall'] > 250).astype(int) +
    (df['river_level'] > 5).astype(int) +
    (df['humidity'] > 85).astype(int)
)

df['risk_level'] = np.where(
    risk_score >= 2, 'High',
    np.where(risk_score == 1, 'Moderate', 'Low')
)

# Output to JSON for dashboard
output = df[['location', 'risk_level']].to_dict(orient='records')

with open("predictions.json", "w") as f:
    json.dump(output, f, indent=4)

print("predictions.json created successfully!")


predictions.json created successfully!


In [8]:
pd.DataFrame(output).head()


,location,risk_level
0,Hyderabad,Low
1,Vijayawada,Low
2,Chennai,High
3,Mumbai,High
4,Bangalore,Low
